In [14]:
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlowCPU/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlowCPU/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TensorFlowCPU/lib/python2.7/site-packages/IPython/extensions')

In [2]:
import pandas as pd
from pandas import DataFrame as df
import json
import re
pd.set_option('display.max_columns', -1)
pd.options.display.max_rows = 4000

In [3]:
dir = ('/vol/medic02/users/ag6516/brain_ct/data/')

In [4]:
annotations = [json.loads(line.rstrip('\n')) for line in open('Brain_CTs_Classification.json')]

In [5]:
def get_df_row(ann):
    ann_id = ann['content'].split(' ')[0].rstrip(':')
    assert 'CHNT' in ann_id
    raw_content = ann['content']
    labels = ann['annotation']['labels']
    return ann_id, raw_content, labels

In [6]:
array = [get_df_row(ann) for ann in annotations]
ann_df = df(array, columns = ['id', 'raw_content', 'labels'])

In [12]:
def basic_filt(ann):
    filtered_report = ann.lower()
    filtered_report = filtered_report.replace('.', ' . ')
    filtered_report = filtered_report.replace('authorised', ' authorised')
    return filtered_report

def extract_addendum(ann):
    match = re.search('(?<=addendum :)(.*$)', ann)
    if match:
        addendum = match.group(1)
        original_report = re.search('(.*?)(?= addendum :)', ann).group(1)
    else:
        addendum = 'None'
        original_report = ann

    return original_report, addendum

def extract_reporter(ann):
    match = re.search('(?<=dictated by)(.*)(?=authorised)', ann)
    if match:
        reporter = match.group(1)
    else:
        reporter = 'None'
    return reporter

def strip_original_report(ann):
    '''
        Strips the CT Head report from the rest of the information.
        If the report is made up of multiple rerpots pertaining to
        multiple CTs, and the reports are not split up by which CT
        they are refering to, return 'None'
    '''
    if 'ct head' in ann:
        if len(re.findall('ct head', ann)) == 1:
            match = re.search('(?<=ct head)(.*)(?=dictated)', ann)
            if match:
                stripped_report = match.group(1)
            else:
                stripped_report = 'None'
        elif len(re.findall('ct head', ann)) > 1:
            match = re.search('(?<=ct head)(.*)(ct head)(.*)(?=cta:)', ann)
            if match:
                stripped_report = match.group(3)
            else:
                stripped_report = 'None'
    else:
        stripped_report = ann
    return stripped_report
        
def strip_addendum(ann):
    match = re.search('(.*)(?=dictated)', ann)
    if match:
        stripped_addendum = match.group(1)
    else:
        stripped_addendum = 'None'
    return stripped_addendum

In [ ]:
# test_report = ann_df.iloc[1]['raw_content']
# test_report = test_report.lower()
# original_report, addendum = extract_addendum(test_report.lower())
# print('Test report \n' + test_report + '\n')

# print('Original report \n' + original_report + '\n')
# original_reporter = extract_reporter(original_report)
# print('Original reporter \n' + original_reporter + '\n')
# stripped_report = strip_original_report(original_report)
# print('Stripped original report \n' + stripped_report + '\n')

# print('Addendum \n' + addendum + '\n')
# addendum_reporter = extract_reporter(addendum)
# print('Addendum reporter \n' + addendum_reporter + '\n')
# stripped_addendum = strip_addendum(addendum)
# print('Stripped addendum \n' + stripped_addendum + '\n')

In [ ]:
def filter_content(content):
#     # replace punctuation
#     for char in ['~', '"']:
#         filt_report = filt_report.replace(char, '')
        
#     for char in ['!', '?', ';', ':', '.']:
#         filt_report = filt_report.replace(char, ' . ')
    
#     for char in ['(', ')', ',', '/', '\\']:
#         filt_report = filt_report.replace(char, '')
    
#     # sentence split
#     filt_content = filt_report.split('.')
    
#     # remove id
#     #filt_content = filt_content[1:]
#     return filt_content

In [13]:
ann_df['basic_filt'] = ann_df.apply(lambda row: basic_filt(row['raw_content']), axis=1)
ann_df['original_report'] = ann_df.apply(lambda row: extract_addendum(row['basic_filt'])[0], axis=1)
ann_df['stripped_original_report'] = ann_df.apply(lambda row: strip_original_report(row['original_report']), axis=1)
ann_df['original_reporter'] = ann_df.apply(lambda row: extract_reporter(row['original_report']), axis=1)
ann_df['addendum'] = ann_df.apply(lambda row: extract_addendum(row['basic_filt'].lower())[1], axis=1)
ann_df['stripped_addendum'] = ann_df.apply(lambda row: strip_addendum(row['addendum']), axis=1)
ann_df['addendum_reporter'] = ann_df.apply(lambda row: extract_reporter(row['addendum']), axis=1)

In [10]:
ann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 10 columns):
id                          13 non-null object
raw_content                 13 non-null object
labels                      13 non-null object
basic_filt                  13 non-null object
original_report             13 non-null object
stripped_original_report    13 non-null object
original_reporter           13 non-null object
addendum                    13 non-null object
stripped_addendum           13 non-null object
addendum_reporter           13 non-null object
dtypes: object(10)
memory usage: 1.1+ KB


In [9]:
ann_df.to_csv('processed_reports.csv')

In [ ]:
ann_df.iloc[0]['raw_content'].replace('.', ' . ')